###  Module & Utility Imports
#### Note: “In the revised version of the paper, MS has been replaced with Shape Factorization (SF).”

In [1]:
import pandas as pd
import numpy as np
import re

### Comparison between the main model and benchmark models w/ SF (Main manuscript)

In [2]:
# === Load CSV ===
df = pd.read_csv('./EvalResults/SummaryTables/BenchCompTabs.csv')
df = df.rename(columns={'MS':'SF', 'MP':'SP'})

# === 모델명 정규화 ===
df["Model"] = df["Model"].str.replace(r".*SKZFC.*", "Ours", regex=True)

def normalize_model(s: str) -> str:
    if s.startswith("ConVAE"):   return "C-VAE"
    if s.startswith("FACVAE"):   return "FAC-VAE"
    if s.startswith("TCVAE"):    return "TC-VAE"
    if s.startswith("VDVAE"):    return "VD-VAE"
    if s.startswith("DiffWave"): return "DiffWave"
    if s.startswith("VDWave"):   return "VDWave"
    if s.startswith("Wavenet"):  return "WaveNet"
    return s

df["Model"] = df["Model"].map(normalize_model)

# === Type 값 변경: ART -> ABP, II -> ECG ===
df["Type"] = df["Type"].replace({"ART": "ABP", "II": "ECG"})

# === Metric Label ===
metric_map = {"fft":"Fast Fourier Transform","matching_pursuit":"Discrete Cosine Transform","welch_evo":"Welch-based Spectral Evolution"}
df["MetricLabel"] = df["MetricType"].map(metric_map)

# === Source × Type 조합별 Subtable ===
source_list = df["Source"].dropna().unique()
type_list   = df["Type"].dropna().unique()
metric_order = ["Fast Fourier Transform","Discrete Cosine Transform","Welch-based Spectral Evolution"]
score_cols = ["ISCOREhm","ISCOREgm", "ISCOREam"]

# 컬럼 구조 생성
col_tuples = [("Dataset – Signal","Model")]
for met in metric_order:
    for sc in score_cols:
        sc_label = sc.replace("ISCORE","").upper()  # AM, GM, HM
        col_tuples.append((met, sc_label))

num_expected_cols = len(col_tuples)

all_rows = []
first_table = True

for s in source_list:
    for t in type_list:
        sub = df[(df["Source"]==s) & (df["Type"]==t)].copy()
        if sub.empty:
            continue

        if not first_table:
            all_rows.append([""]*num_expected_cols)

        header_row = [f"\\textbf{{{s} – {t}}}"] + [""]*(num_expected_cols-1)
        all_rows.append(header_row)

        model_order = ["C-VAE","FAC-VAE","TC-VAE","VD-VAE","DiffWave","VDWave","WaveNet","Ours"]
        sub["Model"] = pd.Categorical(sub["Model"], categories=model_order, ordered=True)
        sub = sub.sort_values("Model")

        wide = sub.pivot(index="Model", columns="MetricLabel", values=score_cols)
        wide = wide.swaplevel(0,1,axis=1).sort_index(axis=1, level=0)

        ordered_cols = []
        for met in metric_order:
            for sc in score_cols:
                if (met, sc) in wide.columns:
                    ordered_cols.append((met, sc))
        wide = wide[ordered_cols]

        style_map = {}
        for col in wide.columns:
            vals = wide[col].dropna()
            order = vals.sort_values(ascending=False).index.tolist()
            style_map[col] = {"first": order[0] if len(order)>0 else None,
                              "second": order[1] if len(order)>1 else None}

        for model in wide.index:
            row = [f"\\textbf{{{model}}}" if model=="Ours" else model]
            for col in wide.columns:
                x = wide.loc[model,col]
                if pd.isna(x):
                    row.append("")
                else:
                    xfmt = f"{x:.3f}"
                    if model == style_map[col]["first"]:
                        row.append(f"\\textbf{{{xfmt}}}")
                    elif model == style_map[col]["second"]:
                        row.append(f"\\underline{{{xfmt}}}")
                    else:
                        row.append(xfmt)
            all_rows.append(row)

        first_table = False

combined_df = pd.DataFrame(all_rows, columns=pd.MultiIndex.from_tuples(col_tuples))

# 기존 포맷 유지
col_format = "l|" + "|".join(["c"*len(score_cols)]*len(metric_order))

latex = combined_df.to_latex(
    index=False,
    escape=False,
    multicolumn=True,
    multicolumn_format="c",
    column_format=col_format,
    caption=(
        "\\textbf{ISCORE-based comparison of ABP and ECG synthesis on MIMIC and VitalDB using all metric components.} "
        "Reported metrics include waveform shape factorization (SF~$\\uparrow$), "
        "shape preservation (SP~$\\uparrow$), "
        "amplitude modulation controllability (AC~$\\uparrow$), "
        "spectral similarity (SS~$\\uparrow$) and reconstructive accuracy (RA~$\\uparrow$), "
        "together with three aggregated ISCORE measures—arithmetic mean (AM), geometric mean (GM) and harmonic mean (HM). "
        "For DiffWave, VDWave and WaveNet, SF was excluded from the ISCORE calculation. "
        "Boldface and underlining indicate the best and second-best results, respectively."
    ),
    label="tab:performance_comparison_all"

)

# 위치 지정자 추가
latex = latex.replace("\\begin{table}", "\\begin{table}[!htbp]")

# === FFT/MP/Welch 오른쪽에 수직선 추가 ===
# multicolumn{3}{c}{FFT} → multicolumn{3}{c|}{FFT} 식으로 변경
latex = latex.replace("& \\multicolumn{3}{c}{Fast Fourier Transform}", "& \\multicolumn{3}{c|}{Fast Fourier Transform}")
latex = latex.replace("& \\multicolumn{3}{c}{Matching Pursuit}", "& \\multicolumn{3}{c|}{Matching Pursuit}")
# 마지막 Welch Evolution은 오른쪽에 더 이상 블록 없으므로 굳이 '|' 안 넣어도 무방
# 필요하면 아래 주석 해제
latex = latex.replace("& \\multicolumn{3}{c}{Welch-based Spectral Evolution}", "& \\multicolumn{3}{c}{Welch-based Spectral Evolution}")

# captionsetup 유지
latex_with_setup = "\\captionsetup{font=footnotesize, margin=-12pt}\n" + latex

print(latex_with_setup)


\captionsetup{font=footnotesize, margin=-12pt}
\begin{table}[!htbp]
\caption{\textbf{ISCORE-based comparison of ABP and ECG synthesis on MIMIC and VitalDB using all metric components.} Reported metrics include waveform shape factorization (SF~$\uparrow$), shape preservation (SP~$\uparrow$), amplitude modulation controllability (AC~$\uparrow$), spectral similarity (SS~$\uparrow$) and reconstructive accuracy (RA~$\uparrow$), together with three aggregated ISCORE measures—arithmetic mean (AM), geometric mean (GM) and harmonic mean (HM). For DiffWave, VDWave and WaveNet, SF was excluded from the ISCORE calculation. Boldface and underlining indicate the best and second-best results, respectively.}
\label{tab:performance_comparison_all}
\begin{tabular}{l|ccc|ccc|ccc}
\toprule
Dataset – Signal & \multicolumn{3}{c|}{Fast Fourier Transform} & \multicolumn{3}{c}{Discrete Cosine Transform} & \multicolumn{3}{c}{Welch-based Spectral Evolution} \\
Model & HM & GM & AM & HM & GM & AM & HM & GM & AM \

### Comparison between the main model and benchmark models w/o SF (Main manuscript)

In [3]:
import pandas as pd
import re

# === Load CSV ===
df = pd.read_csv('./EvalResults/SummaryTables/BenchCompTabs_NOMS.csv')
df = df.rename(columns={'MS':'SF', 'MP':'SP'})

# === 모델명 정규화 ===
df["Model"] = df["Model"].str.replace(r".*SKZFC.*", "Ours", regex=True)

def normalize_model(s: str) -> str:
    if s.startswith("ConVAE"):   return "C-VAE"
    if s.startswith("FACVAE"):   return "FAC-VAE"
    if s.startswith("TCVAE"):    return "TC-VAE"
    if s.startswith("VDVAE"):    return "VD-VAE"
    if s.startswith("DiffWave"): return "DiffWave"
    if s.startswith("VDWave"):   return "VDWave"
    if s.startswith("Wavenet"):  return "WaveNet"
    return s

df["Model"] = df["Model"].map(normalize_model)

# === Type 값 변경: ART -> ABP, II -> ECG ===
df["Type"] = df["Type"].replace({"ART": "ABP", "II": "ECG"})

# === Metric Label ===
metric_map = {"fft":"Fast Fourier Transform","matching_pursuit":"Discrete Cosine Transform","welch_evo":"Welch-based Spectral Evolution"}
df["MetricLabel"] = df["MetricType"].map(metric_map)

# === Source × Type 조합별 Subtable ===
source_list = df["Source"].dropna().unique()
type_list   = df["Type"].dropna().unique()
metric_order = ["Fast Fourier Transform","Discrete Cosine Transform","Welch-based Spectral Evolution"]
score_cols = ["ISCOREhm","ISCOREgm","ISCOREam"]

# 컬럼 구조 생성
col_tuples = [("Dataset – Signal","Model")]
for met in metric_order:
    for sc in score_cols:
        sc_label = sc.replace("ISCORE","").upper()  # AM, GM, HM
        col_tuples.append((met, sc_label))

num_expected_cols = len(col_tuples)

all_rows = []
first_table = True

for s in source_list:
    for t in type_list:
        sub = df[(df["Source"]==s) & (df["Type"]==t)].copy()
        if sub.empty:
            continue

        if not first_table:
            all_rows.append([""]*num_expected_cols)

        header_row = [f"\\textbf{{{s} – {t}}}"] + [""]*(num_expected_cols-1)
        all_rows.append(header_row)

        model_order = ["C-VAE","FAC-VAE","TC-VAE","VD-VAE","DiffWave","VDWave","WaveNet","Ours"]
        sub["Model"] = pd.Categorical(sub["Model"], categories=model_order, ordered=True)
        sub = sub.sort_values("Model")

        wide = sub.pivot(index="Model", columns="MetricLabel", values=score_cols)
        wide = wide.swaplevel(0,1,axis=1).sort_index(axis=1, level=0)

        ordered_cols = []
        for met in metric_order:
            for sc in score_cols:
                if (met, sc) in wide.columns:
                    ordered_cols.append((met, sc))
        wide = wide[ordered_cols]

        style_map = {}
        for col in wide.columns:
            vals = wide[col].dropna()
            order = vals.sort_values(ascending=False).index.tolist()
            style_map[col] = {"first": order[0] if len(order)>0 else None,
                              "second": order[1] if len(order)>1 else None}

        for model in wide.index:
            row = [f"\\textbf{{{model}}}" if model=="Ours" else model]
            for col in wide.columns:
                x = wide.loc[model,col]
                if pd.isna(x):
                    row.append("")
                else:
                    xfmt = f"{x:.3f}"
                    if model == style_map[col]["first"]:
                        row.append(f"\\textbf{{{xfmt}}}")
                    elif model == style_map[col]["second"]:
                        row.append(f"\\underline{{{xfmt}}}")
                    else:
                        row.append(xfmt)
            all_rows.append(row)

        first_table = False

combined_df = pd.DataFrame(all_rows, columns=pd.MultiIndex.from_tuples(col_tuples))

# 기존 포맷 유지
col_format = "l|" + "|".join(["c"*len(score_cols)]*len(metric_order))

latex = combined_df.to_latex(
    index=False,
    escape=False,
    multicolumn=True,
    multicolumn_format="c",
    column_format=col_format,
caption=(
        "\\textbf{ISCORE-based comparison of ABP and ECG synthesis on MIMIC and VitalDB without waveform shape factorization (SF).} "
        "Reported metrics include shape preservation (SP~$\\uparrow$), "
        "amplitude modulation controllability (AC~$\\uparrow$), "
        "spectral similarity (SS~$\\uparrow$) and reconstructive accuracy (RA~$\\uparrow$), "
        "together with three aggregated ISCORE measures—arithmetic mean (AM), geometric mean (GM) and harmonic mean (HM). "
        "Boldface and underlining indicate the best and second-best results, respectively."
    ),
    label="tab:performance_comparison_noSF"
)

# 위치 지정자 추가
latex = latex.replace("\\begin{table}", "\\begin{table}[!htbp]")

# === FFT/MP/Welch 오른쪽에 수직선 추가 ===
# multicolumn{3}{c}{FFT} → multicolumn{3}{c|}{FFT} 식으로 변경
latex = latex.replace("& \\multicolumn{3}{c}{Fast Fourier Transform}", "& \\multicolumn{3}{c|}{Fast Fourier Transform}")
latex = latex.replace("& \\multicolumn{3}{c}{Matching Pursuit}", "& \\multicolumn{3}{c|}{Matching Pursuit}")
# 마지막 Welch Evolution은 오른쪽에 더 이상 블록 없으므로 굳이 '|' 안 넣어도 무방
# 필요하면 아래 주석 해제
latex = latex.replace("& \\multicolumn{3}{c}{Welch-based Spectral Evolution}", "& \\multicolumn{3}{c}{Welch-based Spectral Evolution}")

# captionsetup 유지
latex_with_setup = "\\captionsetup{font=footnotesize, margin=-12pt}\n" + latex

print(latex_with_setup)


\captionsetup{font=footnotesize, margin=-12pt}
\begin{table}[!htbp]
\caption{\textbf{ISCORE-based comparison of ABP and ECG synthesis on MIMIC and VitalDB without waveform shape factorization (SF).} Reported metrics include shape preservation (SP~$\uparrow$), amplitude modulation controllability (AC~$\uparrow$), spectral similarity (SS~$\uparrow$) and reconstructive accuracy (RA~$\uparrow$), together with three aggregated ISCORE measures—arithmetic mean (AM), geometric mean (GM) and harmonic mean (HM). Boldface and underlining indicate the best and second-best results, respectively.}
\label{tab:performance_comparison_noSF}
\begin{tabular}{l|ccc|ccc|ccc}
\toprule
Dataset – Signal & \multicolumn{3}{c|}{Fast Fourier Transform} & \multicolumn{3}{c}{Discrete Cosine Transform} & \multicolumn{3}{c}{Welch-based Spectral Evolution} \\
Model & HM & GM & AM & HM & GM & AM & HM & GM & AM \\
\midrule
\textbf{Mimic – ABP} &  &  &  &  &  &  &  &  &  \\
C-VAE & 0.238 & 0.415 & 0.574 & 0.133 & 0.346 & 

### Comparison between the main model and benchmark models (Supplementary file)

In [4]:
import pandas as pd
import re

# === Load CSV ===
df = pd.read_csv('./EvalResults/SummaryTables/BenchCompTabs.csv')
df_noms = pd.read_csv('./EvalResults/SummaryTables/BenchCompTabs_NOMS.csv')  # (MS 없는 버전)

df = df.rename(columns={'MS':'SF', 'MP':'SP'})
df_noms = df_noms.rename(columns={'MS':'SF', 'MP':'SP'})


# === 모델명 정규화 ===
df["Model"] = df["Model"].str.replace(r".*SKZFC.*", "Ours", regex=True)

def normalize_model(s: str):
    if isinstance(s, float):
        return s
    if s.startswith("ConVAE"):   return "C-VAE"
    if s.startswith("FACVAE"):   return "FAC-VAE"
    if s.startswith("TCVAE"):    return "TC-VAE"
    if s.startswith("VDVAE"):    return "VD-VAE"
    if s.startswith("DiffWave"): return "DiffWave"
    if s.startswith("VDWave"):   return "VDWave"
    if s.startswith("Wavenet"):  return "WaveNet"
    return s

df["Model"] = df["Model"].map(normalize_model)
df["Type"] = df["Type"].replace({"ART": "ABP", "II": "ECG"})
metric_map = {"fft":"Fast Fourier Transform","matching_pursuit":"Discrete Cosine Transform","welch_evo":"Welch-based Spectral Evolution"}
df["MetricLabel"] = df["MetricType"].map(metric_map)

detail_cols = ["SF", "SP", "AC", "SS", "RA"]
iscore_cols = ["ISCOREhm", "ISCOREgm", "ISCOREam"]
all_metric_cols = detail_cols + iscore_cols

# df_noms 동일 전처리
df_noms["Model"] = df_noms["Model"].str.replace(r".*SKZFC.*", "Ours", regex=True)
df_noms["Model"] = df_noms["Model"].map(normalize_model)
df_noms["Type"]  = df_noms["Type"].replace({"ART": "ABP", "II": "ECG"})
df_noms["MetricLabel"] = df_noms["MetricType"].map(metric_map)
for c in ["Source","Type","MetricLabel","Model"]:
    if c not in df_noms.columns:
        df_noms[c] = pd.NA

def lookup_noms_values(source, typ, metric_label):
    sub = df_noms[
        (df_noms["Source"]==source) &
        (df_noms["Type"]==typ) &
        (df_noms["MetricLabel"]==metric_label) &
        (df_noms["Model"]=="Ours")
    ]
    if sub.empty:
        return {}
    r = sub.iloc[0]
    return {
        "ISCOREam": r["ISCOREam"] if "ISCOREam" in r and pd.notna(r["ISCOREam"]) else None,
        "ISCOREgm": r["ISCOREgm"] if "ISCOREgm" in r and pd.notna(r["ISCOREgm"]) else None,
        "ISCOREhm": r["ISCOREhm"] if "ISCOREhm" in r and pd.notna(r["ISCOREhm"]) else None,
    }

source_list = df["Source"].dropna().unique()
type_list   = df["Type"].dropna().unique()
metric_type_order = ["Fast Fourier Transform", "Discrete Cosine Transform", "Welch-based Spectral Evolution"]
model_order = ["C-VAE","FAC-VAE","TC-VAE","VD-VAE","DiffWave","VDWave","WaveNet","Ours"]

def create_table_for_metric_type(metric_type, metric_df):
    all_rows = []
    first_dataset = True
    
    for s in source_list:
        for t in type_list:
            sub = metric_df[(metric_df["Source"]==s) & (metric_df["Type"]==t)].copy()
            if sub.empty:
                continue

            if not first_dataset:
                all_rows.append([""] * (1 + len(all_metric_cols)))

            dataset_header = [f"\\textbf{{{s} – {t}}}"] + [""] * len(all_metric_cols)
            all_rows.append(dataset_header)

            sub["Model"] = pd.Categorical(sub["Model"], categories=model_order, ordered=True)
            sub = sub.sort_values("Model")

            style_map = {}
            for col in all_metric_cols:
                if col in sub.columns:
                    vals = sub[col].dropna()
                    order_idx = vals.sort_values(ascending=False).index.tolist()
                    style_map[col] = {
                        "first": sub.loc[order_idx[0], "Model"] if len(order_idx) > 0 else None,
                        "second": sub.loc[order_idx[1], "Model"] if len(order_idx) > 1 else None
                    }

            noms_iscores = lookup_noms_values(s, t, metric_type)

            for _, row_data in sub.iterrows():
                model = row_data["Model"]
                model_cell = "\\makecell[l]{\\textbf{Ours}\\\\\\footnotesize(no SF)}" if model == "Ours" else model
                row = [model_cell]

                # 상세 Metrics
                for col in detail_cols:
                    if col in row_data and pd.notna(row_data[col]):
                        val_fmt = f"{row_data[col]:.3f}"
                        if col in style_map and model == style_map[col]["first"]:
                            row.append(f"\\textbf{{{val_fmt}}}")
                        elif col in style_map and model == style_map[col]["second"]:
                            row.append(f"\\underline{{{val_fmt}}}")
                        else:
                            row.append(val_fmt)
                    else:
                        row.append("-")

                # ISCORE 컬럼
                for col in iscore_cols:
                    if col in row_data and pd.notna(row_data[col]):
                        main_val = f"{row_data[col]:.3f}"
                        if col in style_map and model == style_map[col]["first"]:
                            main_val_styled = f"\\textbf{{{main_val}}}"
                        elif col in style_map and model == style_map[col]["second"]:
                            main_val_styled = f"\\underline{{{main_val}}}"
                        else:
                            main_val_styled = main_val
                    else:
                        main_val_styled = "-"

                    if model == "Ours":
                        noms_val = noms_iscores.get(col, None)
                        if noms_val is not None:
                            cell = f"\\makecell{{{main_val_styled}\\\\\\footnotesize({noms_val:.3f})}}"
                        else:
                            cell = main_val_styled
                        row.append(cell)
                    else:
                        row.append(main_val_styled)

                all_rows.append(row)
            first_dataset = False

    return all_rows

all_latex_tables = []

for metric_type in metric_type_order:
    metric_df = df[df["MetricLabel"] == metric_type].copy()
    if metric_df.empty:
        continue
    
    table_rows = create_table_for_metric_type(metric_type, metric_df)
    
    # === 헤더 MultiIndex: Metrics / ISCORE 분리 유지 ===
    col_tuples = [("Dataset – Signal", "Model")]
    for col in detail_cols:
        col_tuples.append(("Metrics", col))
    for col in ["HM", "GM", "AM"]:
        col_tuples.append(("ISCORE", col))
    columns = pd.MultiIndex.from_tuples(col_tuples)

    result_df = pd.DataFrame(table_rows, columns=columns)
    col_format = "l|ccccc|ccc"

    latex = result_df.to_latex(
        index=False,
        escape=False,
        multicolumn=True,
        multicolumn_format="c",
        column_format=col_format,
        caption=(
        f"\\textbf{{Detailed comparison of ABP and ECG synthesis on MIMIC and VitalDB for {metric_type}.}} "
        "Reported metrics include waveform shape factorization (SF~$\\uparrow$), "
        "shape preservation (SP~$\\uparrow$), "
        "amplitude modulation controllability (AC~$\\uparrow$), "
        "spectral similarity (SS~$\\uparrow$) and reconstructive accuracy (RA~$\\uparrow$), "
        "together with three aggregated ISCORE measures—arithmetic mean (AM), geometric mean (GM) and harmonic mean (HM). "
        "For \\textbf{Ours}, parentheses in the ISCORE columns indicate scores computed without SF. "
        "Boldface and underlining indicate the best and second-best results, respectively."
    ),
        label=f"tab:detailed_metrics_comparison_{metric_type.lower().replace(' ', '_')}"
    )

    latex = latex.replace("\\begin{table}", "\\begin{table}[!htbp]")
    latex = latex.replace("& \\multicolumn{5}{c}{Metrics}", "& \\multicolumn{5}{c|}{Metrics}")
    latex_with_setup = "\\captionsetup{font=footnotesize, margin=-12pt}\n" + latex

    all_latex_tables.append(latex_with_setup)

for i, table in enumerate(all_latex_tables):
    if i > 0:
        print("\n\\clearpage\n")
    print(table)


\captionsetup{font=footnotesize, margin=-12pt}
\begin{table}[!htbp]
\caption{\textbf{Detailed comparison of ABP and ECG synthesis on MIMIC and VitalDB for Fast Fourier Transform.} Reported metrics include waveform shape factorization (SF~$\uparrow$), shape preservation (SP~$\uparrow$), amplitude modulation controllability (AC~$\uparrow$), spectral similarity (SS~$\uparrow$) and reconstructive accuracy (RA~$\uparrow$), together with three aggregated ISCORE measures—arithmetic mean (AM), geometric mean (GM) and harmonic mean (HM). For \textbf{Ours}, parentheses in the ISCORE columns indicate scores computed without SF. Boldface and underlining indicate the best and second-best results, respectively.}
\label{tab:detailed_metrics_comparison_fast_fourier_transform}
\begin{tabular}{l|ccccc|ccc}
\toprule
Dataset – Signal & \multicolumn{5}{c|}{Metrics} & \multicolumn{3}{c}{ISCORE} \\
Model & SF & SP & AC & SS & RA & HM & GM & AM \\
\midrule
\textbf{Mimic – ABP} &  &  &  &  &  &  &  &  \\
C-VAE

### Sensitivity Metric type 에 따른 디테일 버전 (ISCORE 와 COMPONENTS 통합)

In [5]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from textwrap import dedent

# ===== 폰트/행간 옵션 (기존 유지) =====
TABLE_FONTSIZE = "\\footnotesize"
ARRAY_STRETCH = 0.95

# ===== 칼럼 간격 조정 (새로 추가) =====
TABCOLSEP = "2.5pt"  # 기본값은 6pt, 2pt~4pt 사이로 조정 가능

# ===== multirow 세로 중앙 보정 / 행 높이 표준화 =====
MULTIROW_RAISE_EX = -0.8
ROWSTRUT = "\\rule{0pt}{2.2ex}"

# === 로드 ===
df_sens = pd.read_csv('./EvalResults/SummaryTables/SensitivityDetailTabs.csv')
df_sens.loc[df_sens['IscoreType'] =='MS', 'IscoreType'] = 'SF'
df_sens.loc[df_sens['IscoreType'] =='MP', 'IscoreType'] = 'SP'

# === 전처리 매핑 (기존 유지) ===
metric_name_map = {
    "fft": "Fast Fourier Transform",
    "matching_pursuit": "Discrete Cosine Transform",
    "welch_evo": "Welch-based Spectral Evolution",
}

parameter_latex_map = {
    "J": "$J$", "K": "$K$", "L": "$L$", "N": "$N$", "T": "$T$",
    "window_size": "Window Size", "overlap": "Overlap",
    # ▼ 변경: n_components 라벨을 Components -> Metrics 로
    "n_components": "Metrics", "max_iter": "Max Iter", "tol": "Tolerance",
}

# === 필수 컬럼 체크 (기존 유지) ===
required_cols = {
    "Source","Type","Hyperparameter","Setting","IscoreType",
    "mean","n","std","max","min","MetricType"
}
missing = required_cols - set(df_sens.columns)
if missing:
    raise ValueError(f"Missing required columns in CSV: {missing}")

# === Type 표준화: ART->ABP, II->ECG (대소문자 무관) ===
def normalize_signal_type(x):
    s = str(x).strip()
    u = s.upper()
    if u in {"ART", "ABP"}:
        return "ABP"
    if u in {"II", "ECG"}:
        return "ECG"
    return s

df_sens["Type"] = df_sens["Type"].map(normalize_signal_type)

# === 전처리 (기존 유지) ===
df_sens["MetricType"] = df_sens["MetricType"].map(lambda x: metric_name_map.get(x, x))
df_sens["SourceType"] = df_sens["Source"] + " : " + df_sens["Type"]

def normalize_iscore_type(s: str) -> str:
    s = str(s).upper().strip()
    s = s.replace("ISCORE", "").strip()
    repl = {"ISCOREAM":"AM", "ISCOREGM":"GM", "ISCOREHM":"HM"}
    return repl.get(s, s)

df_sens["IscoreType"] = df_sens["IscoreType"].map(normalize_iscore_type)

def latex_escape(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    return (s.replace("&","\\&").replace("%","\\%").replace("#","\\#")
             .replace("_","\\_").replace("{","\\{").replace("}","\\}")
             .replace("~","\\textasciitilde ").replace("^","\\textasciicircum "))

def format_parameter(param: str) -> str:
    if param in parameter_latex_map:
        return parameter_latex_map[param]
    if isinstance(param, str) and len(param) == 1 and param.isalpha():
        return f"${param}$"
    return latex_escape(str(param))

# === 두 줄 표기: mean / (std) ===
def fmt_val_multiline(m, s, bold=False):
    if pd.isna(m):
        return "-"
    mean_str = f"{m:.3f}"
    if bold:
        mean_str = f"\\textbf{{{mean_str}}}"
    if pd.isna(s):
        return "\\begin{tabular}{@{}c@{}} " + mean_str + " \\end{tabular}"
    std_str = f"{s:.3f}"
    # ▼ std는 bold 처리 안함 (제거됨)
    return "\\begin{tabular}{@{}c@{}} " + mean_str + " \\\\ (" + std_str + ") \\end{tabular}"

# === MetricType별 + 지정 키 묶음 집계 ===
def aggregate_by_metric_and_keys(df: pd.DataFrame, metric_type: str, metric_keys):
    df_m = df[df["MetricType"] == metric_type]
    out_list = []
    for key in metric_keys:
        sub = df_m[df_m["IscoreType"] == key].copy()
        if sub.empty:
            continue
        sub["TotalSum"] = sub["mean"] * sub["n"]
        agg = (
            sub.groupby(["Source","Type","Hyperparameter","Setting"], as_index=False)
               .agg(TotalSum=("TotalSum","sum"),
                    n=("n","sum"),
                    std=("std","mean"),
                    max=("max","max"),
                    min=("min","min"))
        )
        agg["mean"] = agg["TotalSum"] / agg["n"]
        agg.drop(columns=["TotalSum"], inplace=True)
        agg["IscoreType"] = key
        out_list.append(agg)

    if not out_list:
        return pd.DataFrame(columns=["Source","Type","Hyperparameter","Setting","n","std","max","min","mean","IscoreType","SourceType"])

    out = pd.concat(out_list, ignore_index=True)
    out["SourceType"] = out["Source"] + " : " + out["Type"]
    preferred_order = ["MIMIC : ABP","MIMIC : ECG","VitalDB : ABP","VitalDB : ECG"]
    present = [st for st in preferred_order if st in out["SourceType"].unique().tolist()]
    others  = [st for st in out["SourceType"].unique().tolist() if st not in present]
    col_labels = present + others
    out["SourceType"] = pd.Categorical(out["SourceType"], categories=col_labels, ordered=True)
    out.sort_values(["SourceType","Hyperparameter","Setting","IscoreType"], inplace=True)
    return out

# === 파라미터별로 multirow 분리하는 통합 테이블 빌더 (수정됨) ===
def build_table_unified(metric_keys, sens_agg, method_name):
    """
    각 (Source, Type) 그룹당 Dataset/Signal을 한 번만, 중간 행에 표시
    """
    if sens_agg.empty:
        return ""
    exist = [k for k in metric_keys if k in sens_agg["IscoreType"].unique().tolist()]
    if not exist:
        return ""

    groups = sens_agg.groupby("SourceType")
    all_rows = []
    prev_source = None

    total_cols = 4 + len(exist)
    type_rule_full = f"\\cmidrule(lr){{1-{total_cols}}}"
    param_rule_from_param = f"\\cmidrule(lr){{3-{total_cols}}}"

    for source_type, group in groups:
        source, signal_type = source_type.split(" : ")

        if prev_source is not None and source != prev_source:
            all_rows.append("\\midrule")
        elif prev_source is not None:
            all_rows.append(type_rule_full)
        prev_source = source

        block = group[group["IscoreType"].isin(exist)].copy()
        piv = block.pivot_table(
            index=["Hyperparameter","Setting"],
            columns="IscoreType",
            values=["mean","std"],
            aggfunc="first"
        )
        cols_present = [c for c in exist if ("mean", c) in piv.columns]
        if not cols_present:
            continue

        hp_set_idx = list(piv.index)
        total_rows_in_group = len(hp_set_idx)  # ← 전체 행 수
        middle_row_idx = total_rows_in_group // 2  # ← 중간 행 인덱스

        param_groups = {}
        for i, (hp, st) in enumerate(hp_set_idx):
            if hp not in param_groups:
                param_groups[hp] = []
            param_groups[hp].append(i)

        bold_mask_by_col = {c: [False]*len(hp_set_idx) for c in cols_present}
        for c in cols_present:
            for hp, idxs in param_groups.items():
                vals = [(i, piv[("mean", c)].iloc[i]) for i in idxs if pd.notna(piv[("mean", c)].iloc[i])]
                if not vals:
                    continue
                best_i = max(vals, key=lambda t: t[1])[0]
                bold_mask_by_col[c][best_i] = True

        prev_hp = None
        for i, (hp, st) in enumerate(hp_set_idx):
            if prev_hp is not None and hp != prev_hp:
                all_rows.append(param_rule_from_param)

            # ▼ 수정: 첫 행에서만 multirow 생성, raise로 중앙 정렬
            if i == 0:
                # 전체 행의 중앙에 배치되도록 raise 계산
                # (total_rows - 1) / 2.0 만큼 아래로 이동
                VERTICAL_OFFSET = 6.  # ← 이 값을 조정하세요: 양수면 더 아래로, 음수면 위로
                dynamic_raise = MULTIROW_RAISE_EX * ((total_rows_in_group - 1) / 2.0) - VERTICAL_OFFSET
                source_cell = (
                    f"\\multirow[c]{{{total_rows_in_group}}}{{*}}[{dynamic_raise:.2f}ex]"
                    f"{{{latex_escape(source)}}}"
                )
                type_cell = (
                    f"\\multirow[c]{{{total_rows_in_group}}}{{*}}[{dynamic_raise:.2f}ex]"
                    f"{{{latex_escape(signal_type)}}}"
                )
            else:
                source_cell = ""
                type_cell = ""

            prev_hp = hp

            cells = [
                source_cell,
                type_cell,
                "\\multicolumn{1}{c}{" + ROWSTRUT + " " + format_parameter(str(hp)) + "}",
                ROWSTRUT + " " + latex_escape(str(st))
            ]

            for c in metric_keys:
                if c in cols_present:
                    m = piv[("mean", c)].iloc[i]
                    s = piv[("std",  c)].iloc[i] if ("std", c) in piv.columns else np.nan
                    val_tex = fmt_val_multiline(m, s, bold=bold_mask_by_col.get(c, [False]*len(hp_set_idx))[i])
                    cells.append(ROWSTRUT + " " + val_tex)
                else:
                    cells.append(ROWSTRUT + " -")

            all_rows.append(" & ".join(cells) + " \\\\")

    # 헤더 (기존과 동일)
    comp_keys = ["SF","SP","AC","SS","RA"]
    iscore_keys = ["HM","GM","AM"]
    comp_present = [k for k in comp_keys if k in metric_keys]
    iscore_present = [k for k in iscore_keys if k in metric_keys]

    col_format = "cccc|" + "c"*len(comp_present) + "|" + "c"*len(iscore_present)

    header_top = (
        "\\multicolumn{4}{c|}{} & "
        f"\\multicolumn{{{len(comp_present)}}}{{c|}}{{Metrics}} & "
        f"\\multicolumn{{{len(iscore_present)}}}{{c}}{{ISCORE}} \\\\"
    )
    header_second = (
        "Dataset & Signal & "
        "Parameter & "
        "\\multicolumn{1}{c|}{Setting} & "
        + " & ".join(comp_present + iscore_present)
        + " \\\\"
    )

    header_lines = ["\\toprule", header_top, header_second, "\\midrule"]
    table_content = "\n".join(header_lines + all_rows + ["\\bottomrule"])

    label_suffix = method_name.lower().replace(" ", "_")
    cap = (
    f"\\textbf{{Hyperparameter sensitivity analysis for {method_name}.}} "
    "Reported metrics include waveform shape factorization (SF~$\\uparrow$), "
    "shape preservation (SP~$\\uparrow$), "
    "amplitude modulation controllability (AC~$\\uparrow$), "
    "spectral similarity (SS~$\\uparrow$) and reconstructive accuracy (RA~$\\uparrow$), "
    "together with three aggregated ISCORE measures—arithmetic mean (AM), geometric mean (GM) and harmonic mean (HM). "
    "Results are presented as mean~(s.d.) across independent experiments for each parameter setting. "
    "Boldface values indicate the best-performing setting within each parameter group."
        )
    label = f"tab:sensitivity_unified_{label_suffix}"

    fontsize_cmd = (TABLE_FONTSIZE + "\n") if TABLE_FONTSIZE else ""
    arraystretch_cmd = f"\\renewcommand{{\\arraystretch}}{{{ARRAY_STRETCH}}}\n" if ARRAY_STRETCH != 1.0 else ""
    # ▼ 칼럼 간격 조정 추가
    tabcolsep_cmd = f"\\setlength{{\\tabcolsep}}{{{TABCOLSEP}}}\n" if TABCOLSEP else ""

    return "\n".join([
        "\\captionsetup{font=footnotesize, margin=-12pt}",
        "\\begin{table}[!htbp]",
        "\\centering",
        fontsize_cmd + arraystretch_cmd + tabcolsep_cmd,  # ← 칼럼 간격 명령 추가
        f"\\caption{{{cap}}}",
        f"\\label{{{label}}}",
        f"\\begin{{tabular}}{{{col_format}}}",
        table_content,
        "\\end{tabular}",
        "\\end{table}"
    ])

# === 통합 컬럼 순서 & 출력 ===
unified_keys = ["SF","SP","AC","SS","RA","HM","GM","AM"]
method_order = ["Fast Fourier Transform", "Discrete Cosine Transform", "Welch-based Spectral Evolution"]

latex_blocks = []
for method in method_order:
    sens_agg_unified = aggregate_by_metric_and_keys(df_sens, method, unified_keys)
    t_unified = build_table_unified(unified_keys, sens_agg_unified, method_name=method)
    if t_unified:
        latex_blocks.append(t_unified)

def join_with_clearpages(blocks):
    return ("\n\n\\clearpage\n\n").join([b for b in blocks if b])

print("% --- Auto-generated unified sensitivity tables (parameter-wise multirow) ---")
print(join_with_clearpages(latex_blocks))

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4676\4178016876.py:142: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = sens_agg.groupby("SourceType")
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4676\4178016876.py:142: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = sens_agg.groupby("SourceType")
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4676\4178016876.py:142: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future de

% --- Auto-generated unified sensitivity tables (parameter-wise multirow) ---
\captionsetup{font=footnotesize, margin=-12pt}
\begin{table}[!htbp]
\centering
\footnotesize
\renewcommand{\arraystretch}{0.95}
\setlength{\tabcolsep}{2.5pt}

\caption{\textbf{Hyperparameter sensitivity analysis for Fast Fourier Transform.} Reported metrics include waveform shape factorization (SF~$\uparrow$), shape preservation (SP~$\uparrow$), amplitude modulation controllability (AC~$\uparrow$), spectral similarity (SS~$\uparrow$) and reconstructive accuracy (RA~$\uparrow$), together with three aggregated ISCORE measures—arithmetic mean (AM), geometric mean (GM) and harmonic mean (HM). Results are presented as mean~(s.d.) across independent experiments for each parameter setting. Boldface values indicate the best-performing setting within each parameter group.}
\label{tab:sensitivity_unified_fast_fourier_transform}
\begin{tabular}{cccc|ccccc|ccc}
\toprule
\multicolumn{4}{c|}{} & \multicolumn{5}{c|}{Metrics

### Ablation studies for ABP and ECG synthesis on MIMIC and VitalDB

In [6]:
import pandas as pd
import re

# === Load CSV ===
df = pd.read_csv('./EvalResults/SummaryTables/AblCompTabs.csv')
df = df.rename(columns={'MS':'SF', 'MP':'SP'})

# === 모델명 정규화 ===
def normalize_model(s: str):
    if isinstance(s, float):
        return s
    if "SKZFC_" in s:
        return "Ours"
    if "FC_" in s:
        return "$only-\\theta-prior$"
    if "SKZ_" in s:
        return "$only-z-prior$"
    return s

df["Model"] = df["Model"].map(normalize_model)

# === 데이터 전처리 ===
df["Type"] = df["Type"].replace({"ART": "ABP", "II": "ECG"})
metric_map = {
    "fft": "Fast Fourier Transform",
    "matching_pursuit": "Discrete Cosine Transform",
    "welch_evo": "Welch-based Spectral Evolution"
}
df["MetricLabel"] = df["MetricType"].map(metric_map)

# === 메트릭 컬럼 정의 ===
detail_cols = ["SF", "SP", "AC", "SS", "RA"]
iscore_cols = ["ISCOREhm", "ISCOREgm", "ISCOREam"]
all_metric_cols = detail_cols + iscore_cols

# === 고유값 추출 ===
source_list = df["Source"].dropna().unique()
type_list = df["Type"].dropna().unique()
metric_type_order = ["Fast Fourier Transform", "Discrete Cosine Transform", "Welch-based Spectral Evolution"]

# 모델 순서 정의
model_order = ["$only-\\theta-prior$", "$only-z-prior$", "Ours"]

def create_table_for_metric_type(metric_type, metric_df):
    all_rows = []
    first_dataset = True
    
    for s in source_list:
        for t in type_list:
            sub = metric_df[(metric_df["Source"] == s) & (metric_df["Type"] == t)].copy()
            if sub.empty:
                continue

            # 데이터셋 구분선
            if not first_dataset:
                all_rows.append([""] * (1 + len(all_metric_cols)))

            # 데이터셋 헤더
            dataset_header = [f"\\textbf{{{s} – {t}}}"] + [""] * len(all_metric_cols)
            all_rows.append(dataset_header)

            # 모델 순서대로 정렬
            sub["Model"] = pd.Categorical(sub["Model"], categories=model_order, ordered=True)
            sub = sub.sort_values("Model")

            # 각 메트릭별 최고/차상위 찾기
            style_map = {}
            for col in all_metric_cols:
                if col in sub.columns:
                    vals = sub[col].dropna()
                    if len(vals) > 0:
                        order_idx = vals.sort_values(ascending=False).index.tolist()
                        style_map[col] = {
                            "first": sub.loc[order_idx[0], "Model"] if len(order_idx) > 0 else None,
                            "second": sub.loc[order_idx[1], "Model"] if len(order_idx) > 1 else None
                        }

            # 각 모델별로 행 추가
            for _, row_data in sub.iterrows():
                model = row_data["Model"]
                # Ours 모델명 볼드 처리
                model_cell = f"\\textbf{{{model}}}" if model == "Ours" else model
                row = [model_cell]

                # 상세 Metrics
                for col in detail_cols:
                    if col in row_data and pd.notna(row_data[col]):
                        val_fmt = f"{row_data[col]:.3f}"
                        if col in style_map and model == style_map[col]["first"]:
                            row.append(f"\\textbf{{{val_fmt}}}")
                        elif col in style_map and model == style_map[col]["second"]:
                            row.append(f"\\underline{{{val_fmt}}}")
                        else:
                            row.append(val_fmt)
                    else:
                        row.append("-")

                # ISCORE 컬럼
                for col in iscore_cols:
                    if col in row_data and pd.notna(row_data[col]):
                        val_fmt = f"{row_data[col]:.3f}"
                        if col in style_map and model == style_map[col]["first"]:
                            row.append(f"\\textbf{{{val_fmt}}}")
                        elif col in style_map and model == style_map[col]["second"]:
                            row.append(f"\\underline{{{val_fmt}}}")
                        else:
                            row.append(val_fmt)
                    else:
                        row.append("-")

                all_rows.append(row)
            
            first_dataset = False

    return all_rows

all_latex_tables = []

for metric_type in metric_type_order:
    metric_df = df[df["MetricLabel"] == metric_type].copy()
    if metric_df.empty:
        continue
    
    table_rows = create_table_for_metric_type(metric_type, metric_df)
    
    # === 헤더 MultiIndex: Metrics / ISCORE 분리 ===
    col_tuples = [("Dataset – Signal", "Model")]
    for col in detail_cols:
        col_tuples.append(("Metrics", col))
    for col in ["HM", "GM", "AM"]:
        col_tuples.append(("ISCORE", col))
    columns = pd.MultiIndex.from_tuples(col_tuples)

    result_df = pd.DataFrame(table_rows, columns=columns)
    col_format = "l|ccccc|ccc"

    latex = result_df.to_latex(
        index=False,
        escape=False,
        multicolumn=True,
        multicolumn_format="c",
        column_format=col_format,
        caption = (
    f"\\textbf{{Ablation study of ABP and ECG synthesis on MIMIC and VitalDB for {metric_type}.}} "
    "This table compares three model variants: $only\\text{-}\\theta\\text{-prior}$ (frequency-domain prior only), "
    "$only\\text{-}z\\text{-prior}$ (latent-space prior only), and \\textbf{Ours} (full model with both priors). "
    "Reported metrics include waveform shape factorization (SF~$\\uparrow$), "
    "shape preservation (SP~$\\uparrow$), "
    "amplitude modulation controllability (AC~$\\uparrow$), "
    "spectral similarity (SS~$\\uparrow$) and reconstructive accuracy (RA~$\\uparrow$), "
    "together with three aggregated ISCORE measures—arithmetic mean (AM), geometric mean (GM) and harmonic mean (HM). "
    "Boldface and underlining indicate the best and second-best results, respectively."
    ),
        label=f"tab:ablation_study_{metric_type.lower().replace(' ', '_')}"
    )

    latex = latex.replace("\\begin{table}", "\\begin{table}[!htbp]")
    latex = latex.replace("& \\multicolumn{5}{c}{Metrics}", "& \\multicolumn{5}{c|}{Metrics}")
    latex_with_setup = "\\captionsetup{font=footnotesize, margin=-12pt}\n" + latex

    all_latex_tables.append(latex_with_setup)

# === 출력 ===
for i, table in enumerate(all_latex_tables):
    if i > 0:
        print("\n\\clearpage\n")
    print(table)

\captionsetup{font=footnotesize, margin=-12pt}
\begin{table}[!htbp]
\caption{\textbf{Ablation study of ABP and ECG synthesis on MIMIC and VitalDB for Fast Fourier Transform.} This table compares three model variants: $only\text{-}\theta\text{-prior}$ (frequency-domain prior only), $only\text{-}z\text{-prior}$ (latent-space prior only), and \textbf{Ours} (full model with both priors). Reported metrics include waveform shape factorization (SF~$\uparrow$), shape preservation (SP~$\uparrow$), amplitude modulation controllability (AC~$\uparrow$), spectral similarity (SS~$\uparrow$) and reconstructive accuracy (RA~$\uparrow$), together with three aggregated ISCORE measures—arithmetic mean (AM), geometric mean (GM) and harmonic mean (HM). Boldface and underlining indicate the best and second-best results, respectively.}
\label{tab:ablation_study_fast_fourier_transform}
\begin{tabular}{l|ccccc|ccc}
\toprule
Dataset – Signal & \multicolumn{5}{c|}{Metrics} & \multicolumn{3}{c}{ISCORE} \\
Model &